# Fractional Cover Model Testing with Tensorflow
This notebook trials RIOS and Rasterio versions of the Tensorflow model using both native Tensorflow and the TensorflowLite frameworks.


Peter Scarth 2020-02-05


In [2]:
# Install TFLite if required
# Find the correct version at https://www.tensorflow.org/lite/guide/python
#!pip install https://dl.google.com/coral/python/tflite_runtime-2.1.0.post1-cp37-cp37m-linux_x86_64.whl
import tflite_runtime.interpreter as tflite


### Convert the TF model to TFLite Format

In [1]:
import tensorflow as tf
print(tf.__version__)

model=tf.keras.models.load_model('fcModel_202004230248.hd5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("fcModel_202004230248.tflite", "wb").write(tflite_model)


2.0.0


6688

## RasterIO with Tensorflow Full

In [3]:
%%time
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import rasterio
import numpy as np
import time
from IPython.display import clear_output

import tensorflow as tf
print(tf.__version__)


# The unmixing model
def unmixfc(nbar, fcModel):
    # Get the shape of the input array
    inshape = nbar.shape
    # Flatten and scale reflectance
    nbar = np.reshape(nbar,(inshape[0],-1)) / 10000.0

    # Convert Landsat 8 if required
    #nbar = np.transpose(np.transpose(nbar) * 
    #                    np.array([0.97470, 0.99779, 1.00446, 0.98906, 0.99467, 1.02551]) + 
    #                    np.array([0.00041, 0.00289, 0.00274, 0.00004, 0.00256, -0.00327]))

    # Convert Sentinel 2 if required - DEA - bands are 1 2 3 4 5 6 7 8 11 12 8a FMASK TOPO
    #nbar = np.transpose(np.transpose(nbar[[1,2,3,7,8,9]]) * 
    #                   np.array([0.9551, 1.0582, 0.9871, 1.0187, 0.9528, 0.9688]) + 
    #                  np.array([-0.0022, 0.0031, 0.0064, 0.012, 0.0079, -0.0042]))

    # Drop the Blue band
    refData = np.transpose(nbar[1:])
    # Run the prediction
    fcLayers = fcModel.predict(refData,batch_size=262144)
    # Clip Output
    outputFC = np.clip(np.round(100 + 100 * fcLayers.T, 0),1,255)
    # Correct noData
    outputFC[:,nbar[0] < 0.001] = 0
    outputFC[:,nbar[0] > 0.999] = 0
    # Reshape the FC Output
    return np.reshape(outputFC,(3,inshape[1],inshape[2])).astype(np.uint8)


# Connect to the seasonal surface reflectance
inFile = 'lztmre_sa_m200703200705_dbia2.tif'
outFile = 'lztmre_sa_m200703200705_dbia2_FC_RA_TF.tif'
fcModel = tf.keras.models.load_model('fcModel20200415.hd5')

    

with rasterio.open(inFile) as src:
    kwargs = src.meta
    kwargs['count']=3
    kwargs['nodata']=0
    kwargs['dtype']='uint8'
    # Tiling iterator
    tiles = src.block_windows(0)
    with rasterio.open(outFile, 'w', **kwargs) as dst:
        # Set Up Timing
        blockCount = 0
        numTiles = len(list(src.block_windows(0)))
        startTime = time.time()

        for idx, window in tiles:
            srcData = src.read(window=window).astype(np.float)
            dstData = unmixfc(srcData,fcModel) # Do the Processing Here
            dst.write( dstData, window=window)
            # Calculate and display timing
            blockCount += 1
            elapsedTime = time.time()-startTime
            remainingTime = (elapsedTime / blockCount) * (numTiles - blockCount )
            clear_output(wait=True)
            print(str(np.round(remainingTime / 60, 2))+' minutes to go')





0.0 minutes to go
CPU times: user 25min 59s, sys: 2min 10s, total: 28min 9s
Wall time: 16min 28s


## RasterIO with Tensorflow Lite


In [1]:
%%time
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import rasterio
import numpy as np
import time
from IPython.display import clear_output

import tflite_runtime.interpreter as tflite




# The unmixing model
def unmixfc(nbar):
    # Get the shape of the input array
    inshape = nbar.shape
    # Flatten and scale reflectance
    nbar = np.reshape(nbar,(inshape[0],-1)) / 10000.0

    # Convert Landsat 8 if required
    #nbar = np.transpose(np.transpose(nbar) * 
    #                    np.array([0.97470, 0.99779, 1.00446, 0.98906, 0.99467, 1.02551]) + 
    #                    np.array([0.00041, 0.00289, 0.00274, 0.00004, 0.00256, -0.00327]))

    # Convert Sentinel 2 if required - DEA - bands are 1 2 3 4 5 6 7 8 11 12 8a FMASK TOPO
    #nbar = np.transpose(np.transpose(nbar[[1,2,3,7,8,9]]) * 
    #                   np.array([0.9551, 1.0582, 0.9871, 1.0187, 0.9528, 0.9688]) + 
    #                  np.array([-0.0022, 0.0031, 0.0064, 0.012, 0.0079, -0.0042]))


    # Drop the Blue band
    refData = np.transpose(nbar[1:])
    # Run the prediction in TFLite
    interpreter = tflite.Interpreter(model_path=TFLITE_MODEL)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.resize_tensor_input(input_details[0]['index'], refData.shape)
    interpreter.allocate_tensors()
    interpreter.set_tensor(input_details[0]['index'], refData.astype(np.float32))
    interpreter.invoke()
    fcLayers = interpreter.get_tensor(output_details[0]['index'])


    # Clip Output
    outputFC = np.clip(np.round(100 + 100 * fcLayers.T, 0),1,255)
    # Correct noData
    outputFC[:,nbar[0] < 0.001] = 0
    outputFC[:,nbar[0] > 0.999] = 0
    # Reshape the FC Output
    return np.reshape(outputFC,(3,inshape[1],inshape[2])).astype(np.uint8)


def main(infile,outfile):
    with rasterio.open(inFile) as src:
        kwargs = src.meta
        kwargs['count']=3
        kwargs['nodata']=0
        kwargs['dtype']='uint8'
        # Tiling iterator
        tiles = src.block_windows(0)
        with rasterio.open(outFile, 'w', **kwargs) as dst:
            # Set Up Timing
            blockCount = 0
            numTiles = len(list(src.block_windows(0)))
            startTime = time.time()

            for idx, window in tiles:
                srcData = src.read(window=window).astype(np.float32)
                dstData = unmixfc(srcData) # Do the Processing Here
                dst.write( dstData, window=window)
                # Calculate and display timing
                blockCount += 1
                elapsedTime = time.time()-startTime
                remainingTime = (elapsedTime / blockCount) * (numTiles - blockCount )
                clear_output(wait=True)
                print(str(np.round(remainingTime / 60, 2))+' minutes to go')

# Connect to the seasonal surface reflectance
TFLITE_MODEL = "fcModel20200415.tflite"
inFile = 'lztmre_sa_m200703200705_dbia2.tif'
outFile = 'lztmre_sa_m200703200705_dbia2__FC_RA_TFL.tif'
main(inFile,outFile)

0.0 minutes to go
CPU times: user 11min 23s, sys: 2min 37s, total: 14min
Wall time: 16min 5s


## RIOS with Tensorflow Full


In [2]:
%%time
from __future__ import absolute_import, division, print_function, unicode_literals

import os
from rios import applier
import numpy as np
import tensorflow as tf
print(tf.__version__)

def unmixfc(info,inputs,outputs,otherargs):   

    
    nbar = inputs.nbar
    # Get the shape of the input array
    inshape = nbar.shape
    # Flatten and floating point reflectance
    nbar = np.reshape(nbar,(inshape[0],-1)) / 10000.0

    # Convert Landsat 8 if required
    #nbar = np.transpose(np.transpose(nbar) * 
    #                    np.array([0.97470, 0.99779, 1.00446, 0.98906, 0.99467, 1.02551]) + 
    #                    np.array([0.00041, 0.00289, 0.00274, 0.00004, 0.00256, -0.00327]))

    # Drop the Blue band
    refData = np.transpose(nbar[1:])
    # Load the TF session
    fcLayers = otherargs.fcModel.predict(refData,batch_size=262144)
    # Scale Fractions if required
    #totalSum = np.sum(fcLayers,axis=1) + np.finfo('float32').eps
    #outputFC = np.round(100 + 100 * fcLayers.T / totalSum,0)
    # Clip Output
    outputFC = np.clip(np.round(100 + 100 * fcLayers.T, 0),1,255)
    # Correct noData
    outputFC[:,nbar[0] < 0.001] = 0
    outputFC[:,nbar[0] > 0.999] = 0
    # Write the FC Output
    outputFC = np.reshape(outputFC,(3,inshape[1],inshape[2]))
    outputs.fc =  outputFC.astype(np.uint8)


      

def main(infile,outfile):
    
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    infiles.nbar = infile
    outfiles.fc = outfile

    otherargs = applier.OtherInputs()
    otherargs.fcModel = fcModel

    controls = applier.ApplierControls()
    controls.windowxsize = 512
    controls.windowysize = 512
    controls.setStatsIgnore(0)
    #controls.setNumThreads(4)
    #controls.setJobManagerType('multiprocessing')
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                 "ZLEVEL=9",
                                 "BIGTIFF=YES",
                                 "TILED=YES",
                                 "INTERLEAVE=BAND",
                                 "NUM_THREADS=ALL_CPUS",
                                 "BLOCKXSIZE=512",
                                 "BLOCKYSIZE=512"])

    applier.apply(unmixfc, infiles, outfiles, otherargs, controls=controls)


# FIX CONDA PROJ ISSUES
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
#os.environ['PROJ_LIB'] = r'C:\Users\scart\Miniconda3\envs\solaris\Library\share\proj'

# Connect to the seasonal surface reflectance
inFile = 'lztmre_sa_m200703200705_dbia2.tif'
outFile = 'lztmre_sa_m200703200705_dbia2_FC_RI_TF.tif'
fcModel = tf.keras.models.load_model('fcModel20200415.hd5')

main(inFile,outFile)


2.0.0
CPU times: user 42min 17s, sys: 3min 5s, total: 45min 23s
Wall time: 18min 40s


## RIOS with Tensorflow Lite

In [3]:
%%time
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
from rios import applier
import tflite_runtime.interpreter as tflite


def unmixfc(info,inputs,outputs,otherargs):   

    
    nbar = inputs.nbar
    # Get the shape of the input array
    inshape = nbar.shape
    # Flatten and floating point reflectance
    nbar = np.reshape(nbar,(inshape[0],-1)) / 10000.0

    #Convert Landsat 8 if required
    nbar = np.transpose(np.transpose(nbar) * 
                        np.array([0.97470, 0.99779, 1.00446, 0.98906, 0.99467, 1.02551]) + 
                        np.array([0.00041, 0.00289, 0.00274, 0.00004, 0.00256, -0.00327]))

    # Convert Sentinel 2 if required - DEA - bands are 1 2 3 4 5 6 7 8 11 12 8a FMASK TOPO
    #nbar = np.transpose(np.transpose(nbar) * 
    #                   np.array([0.9551, 1.0582, 0.9871, 1.0187, 0.9528, 0.9688]) + 
    #                  np.array([-0.0022, 0.0031, 0.0064, 0.012, 0.0079, -0.0042]))

    # Drop the Blue band
    refData = np.transpose(nbar[1:])
    # Predict using TFLite
    fcModel = tflite.Interpreter(model_path=TFLITE_MODEL)
    inputDetails = fcModel.get_input_details()
    outputDetails = fcModel.get_output_details()
    fcModel.resize_tensor_input(inputDetails[0]['index'], refData.shape)
    fcModel.allocate_tensors()
    fcModel.set_tensor(inputDetails[0]['index'], refData.astype(np.float32))
    fcModel.invoke()
    fcLayers = fcModel.get_tensor(outputDetails[0]['index'])
    # Scale Fractions if required
    #totalSum = np.sum(fcLayers,axis=1) + np.finfo('float32').eps
    #outputFC = np.round(100 + 100 * fcLayers.T / totalSum,0)
    # Clip Output

    fcLayers = np.clip(fcLayers.T,0,2)
    fcLayerSum = fcLayers.sum(axis=0) + np.finfo('float32').eps
    outputFC = np.round(100 * fcLayers/fcLayerSum, 0)
    # Correct noData
    outputFC[:,nbar[0] < 0.001] = 255
    outputFC[:,nbar[0] > 0.999] = 255
    # Write the FC Output
    outputFC = np.reshape(outputFC,(3,inshape[1],inshape[2]))
    outputs.fc =  outputFC.astype(np.uint8)


      
def main(infile,outfile):
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    infiles.nbar = infile
    outfiles.fc = outfile

    otherargs = applier.OtherInputs()
    #otherargs.fcModel = fcModel

    controls = applier.ApplierControls()
    controls.windowxsize = 512
    controls.windowysize = 512
    controls.setStatsIgnore(255)
    controls.setNumThreads(8)
    controls.setJobManagerType('multiprocessing')
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                 "ZLEVEL=9",
                                 "BIGTIFF=YES",
                                 "TILED=YES",
                                 "INTERLEAVE=BAND",
                                 "NUM_THREADS=ALL_CPUS",
                                 "BLOCKXSIZE=512",
                                 "BLOCKYSIZE=512"])

    applier.apply(unmixfc, infiles, outfiles, otherargs, controls=controls)

# FIX CONDA PROJ ISSUES
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
#os.environ['PROJ_LIB'] = r'C:\Users\scart\Miniconda3\envs\solaris\Library\share\proj'

# Connect to the seasonal surface reflectance
#inFile = 'lztmre_sa_m200703200705_dbia2.tif'
#outFile = 'lztmre_sa_m200703200705_dbia2_FC_RI_TFL_3_8_32.tif'
inFile = 'l8olre_aus_m201409201411_dbia2.tif'
outFile = 'l8olre_aus_m201409201411_fc_constrained_32_32_32_a2.tif'
TFLITE_MODEL = "fcModel_202005190514.tflite" # Best So Far Constrained 32 x 32 x 32 4h 42min 57s
#TFLITE_MODEL = "fcModel_202005182351.tflite" # Best So Far Constrained 256_64_256
#TFLITE_MODEL = "fcModel_202005242216.tflite" #256_128_256
#TFLITE_MODEL = "fcModel_202005250053.tflite" #64_64_64


main(inFile,outFile)


CPU times: user 4h 36min 19s, sys: 4min 9s, total: 4h 40min 29s
Wall time: 1h 55min 32s


In [1]:
from shutil import copyfile
copyfile("fcModel_202005190514.hd5", "fcModel_32x32x32.hd5")
copyfile("fcModel_202005182351.hd5", "fcModel_256x64x256.hd5")
copyfile("fcModel_202005242216.hd5", "fcModel_256x128x256.hd5")
copyfile("fcModel_202005250053.hd5", "fcModel_64x64x64.tflite")

'fcModel_64x64x64.tflite'

In [11]:
import numpy as np
import tflite_runtime.interpreter as tflite
TFLITE_MODEL = "fcModel_202005110223.tflite"


    
nbar=np.array([0.02215505, 0.04722694, 0.05210273, 0.2560805,  0.18626416, 0.09355192])
# Convert Sentinel 2 if required - DEA - bands are 1 2 3 4 5 6 7 8 11 12 8a FMASK TOPO
nbar = np.transpose(np.transpose(nbar) * 
                   np.array([0.9551, 1.0582, 0.9871, 1.0187, 0.9528, 0.9688]) + 
                  np.array([-0.0022, 0.0031, 0.0064, 0.012, 0.0079, -0.0042]))

# Drop the Blue band
refData = np.transpose(nbar[1:])
# Predict using TFLite
fcModel = tflite.Interpreter(model_path=TFLITE_MODEL)
inputDetails = fcModel.get_input_details()
outputDetails = fcModel.get_output_details()
fcModel.resize_tensor_input(inputDetails[0]['index'], refData.shape)
fcModel.allocate_tensors()
fcModel.set_tensor(inputDetails[0]['index'], refData.astype(np.float32))
fcModel.invoke()
fcLayers = fcModel.get_tensor(outputDetails[0]['index'])
print(fcLayers)
# Clip the output to positive
fcLayers = np.clip(fcLayers.T,0,2)
# Comput the sum of the three bands
fcLayerSum = fcLayers.sum(axis=0) + np.finfo('float32').eps
# Make the output sum to 100
outputFC = np.round(100 * fcLayers/fcLayerSum, 0)
# Correct noData (TODO ADD IN NODATA VALUE)
outputFC[:,nbar[0] < 0.001] = 255
outputFC[:,nbar[0] > 0.999] = 255

print(outputFC)
       

[[0.00490556 0.6045726  0.3899848 ]]
[[ 0.]
 [60.]
 [39.]]


## Extract

In [14]:
import rasterio
import numpy as np
import fiona
from shapely.geometry import shape
from affine import Affine
import rasterio.features


# open the output file
rasterName = 'lztmre_sa_m200703200705_dbia2.tif'
# The image of interest
dataSet = rasterio.open(rasterName)

# Open the result file
resultFile = open(rasterName+'.csv','w',1)
with fiona.open('bare.shp', "r") as shapeSet:
# Iterate for all features in the shapefile
    for feature in shapeSet:
        # Get pixel coordinates of the geometry's bounding box
        geometry = shape(feature['geometry'])
        ul = dataSet.index(*geometry.bounds[0:2])
        lr = dataSet.index(*geometry.bounds[2:4])
        window = ((lr[0], ul[0]+1), (ul[1], lr[1]+1))
        # create an affine transform for the subset data
        t = dataSet.transform
        shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)

        # rasterize the geometry
        mask = rasterio.features.rasterize(
            [(geometry, 1)],
            out_shape=dataSet.read(1, window=window).shape,
            transform=shifted_affine,
            fill=0,
            all_touched=True,
            dtype=np.uint8)


        # Read the subset of the data into a numpy array
        resultFile.write(str(feature['properties']['fid'])+",")
        for band in range(0, dataSet.count):

            # Read in a subset - rasterio is 1 indexed
            dataBand = dataSet.read(band+1, window=window).astype('float32')


            # Mask the nodata and out of bounds areas
            dataBand[dataBand==0]=np.nan
            dataBand[mask==0]=np.nan

            # Compute the mean, variance etc
            dataMean = np.nanmean(dataBand)

            resultFile.write(str(dataMean)+",")
        
        resultFile.write("\n")



        
resultFile.close()